In [ ]:
from pyspark.sql import Window
from pyspark.sql.functions import col, desc, row_number, first
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType, FloatType

In [ ]:

#Instead of keyvault we are using the id and key directly here
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "",
"fs.azure.account.oauth2.client.secret": '',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/code/oauth2/token"}


dbutils.fs.mount(
source = "abfss://airbnbdata@airbnbprojectpoojap.dfs.core.windows.net", # contrainer@storageacc
mount_point = "/mnt/airbnbmountdata",
extra_configs = configs)
  
#now we check if mount was successful

True

In [ ]:
%fs
ls "/mnt/airbnbmountdata"

path,name,size,modificationTime
dbfs:/mnt/airbnbmountdata/raw-data/,raw-data/,0,1700332641000
dbfs:/mnt/airbnbmountdata/transformed-data/,transformed-data/,0,1700332679000


In [ ]:
listings = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/airbnbmountdata/raw-data/bostonlistings.csv")
reviews = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/airbnbmountdata/raw-data/bostonreviews.csv")

In [ ]:
listings.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+-------------------------+--------------------+--------------------+----------------------+--------------------+----------------------+----------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------------+----------------------+----------------------+-------------

In [ ]:
reviews.show()

+----------+--------+----------+-----------+-------------+--------------------------------+
|listing_id|      id|      date|reviewer_id|reviewer_name|                        comments|
+----------+--------+----------+-----------+-------------+--------------------------------+
|      3781|4.14E+17| 7/25/2021|  387513970|          Ana|            I stayed at this ...|
|      3781|5.51E+17| 1/30/2022|  387833418|        Alexa|            Great location fo...|
|      3781|6.38E+17| 5/29/2022|   51090123|     Carolina|            Great place to sp...|
|      3781|7.09E+17|  9/5/2022|   69554381|       Mhoire|            Great place.  Gre...|
|      5506|4.10E+17| 7/19/2021|  138760242|        David|            Nice place. Close...|
|      5506|4.64E+17| 10/1/2021|  150474842|       Robert|            Cozy little room ...|
|      5506|4.81E+17|10/25/2021|   59941493|     Cristina|            Wonderful little ...|
|      5506|6.58E+17| 6/26/2022|  364272935|        Seong|            Very nice 

In [ ]:
reviews.printSchema()

root
 |-- listing_id: double (nullable = true)
 |-- id: double (nullable = true)
 |-- date: date (nullable = true)
 |-- reviewer_id: integer (nullable = true)
 |-- reviewer_name: string (nullable = true)
 |-- comments: string (nullable = true)



In [ ]:
listings.printSchema()

root
 |-- id: string (nullable = true)
 |-- listing_url: string (nullable = true)
 |-- scrape_id: string (nullable = true)
 |-- last_scraped: string (nullable = true)
 |-- source: string (nullable = true)
 |-- name: string (nullable = true)
 |-- description: string (nullable = true)
 |-- neighborhood_overview: string (nullable = true)
 |-- picture_url: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_url: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- host_since: string (nullable = true)
 |-- host_location: string (nullable = true)
 |-- host_about: string (nullable = true)
 |-- host_response_time: string (nullable = true)
 |-- host_response_rate: string (nullable = true)
 |-- host_acceptance_rate: string (nullable = true)
 |-- host_is_superhost: string (nullable = true)
 |-- host_thumbnail_url: string (nullable = true)
 |-- host_picture_url: string (nullable = true)
 |-- host_neighbourhood: string (nullable = true)
 |-- host_listings_cou

In [ ]:
columns_to_drop = ["scrape_id", "last_scraped", "source"]

listings = listings.drop(*columns_to_drop)


In [ ]:
reviews = reviews.withColumnRenamed("id", "review_id")


In [ ]:
# Get reviews data sorted with latest reviews shwon on top grouped by listing id
reviews = reviews.withColumn("date", col("date").cast("timestamp"))

window_spec = Window.partitionBy("listing_id").orderBy(col("date").desc())

sorted_reviews = reviews.withColumn("latest_review_number", row_number().over(window_spec))

sorted_reviews.show()

+----------+---------+-------------------+-----------+-------------+--------------------------------+--------------------+
|listing_id|review_id|               date|reviewer_id|reviewer_name|                        comments|latest_review_number|
+----------+---------+-------------------+-----------+-------------+--------------------------------+--------------------+
|    6695.0|  9.68E17|2023-08-28 00:00:00|   20655826|        Byron|            Terry and Mindy w...|                   1|
|    6695.0|  8.56E17|2023-03-26 00:00:00|  186132767|        Carly|            We had a great ti...|                   2|
|    6695.0|  7.63E17|2022-11-18 00:00:00|   93697560|      Joaquin|            Great, clean, and...|                   3|
|    6695.0|  7.54E17|2022-11-06 00:00:00|  451130006|       Konomi|            Terry is the swee...|                   4|
|    6695.0|  7.51E17|2022-11-01 00:00:00|  178154261|      Michael|            Place was great! ...|                   5|
|    6695.0|  7.

In [ ]:
# add recent review to listings table

latest_reviews = sorted_reviews.filter(col("latest_review_number") == 1)

latest_reviews.show()


+----------+---------+-------------------+-----------+-------------+--------------------+--------------------+
|listing_id|review_id|               date|reviewer_id|reviewer_name|            comments|latest_review_number|
+----------+---------+-------------------+-----------+-------------+--------------------+--------------------+
|    3781.0|  7.09E17|2022-09-05 00:00:00|   69554381|       Mhoire|Great place.  Gre...|                   1|
|    5506.0|  9.65E17|2023-08-24 00:00:00|  244427805|        Sarah|Enjoyed the small...|                   1|
|    6695.0|  9.68E17|2023-08-28 00:00:00|   20655826|        Byron|Terry and Mindy w...|                   1|
|    8789.0|   9.7E17|2023-08-31 00:00:00|   61665312|      Nicolas|Great location an...|                   1|
|   18711.0|  9.62E17|2023-08-19 00:00:00|  436019467| Alexa Jeanne|My recent stay at...|                   1|
|   29765.0|  7.43E17|2022-10-21 00:00:00|    5152671|        David|"Elizabeth's plac...|                   1|
|

In [ ]:
listings.show()

+--------------------+--------------------+--------------------+--------------------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+-------------------------+--------------------+--------------------+----------------------+--------------------+----------------------+----------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+-------

In [ ]:
# add recent review to the listings table
listings = listings.withColumnRenamed("id", "listing_id")

result_df = listings.join(latest_reviews.select("listing_id", "comments").alias("latest_reviews"),
                           on="listing_id",
                           how="left_outer")


result_df = result_df.drop("listing_id")

result_df.show()

+--------------------+--------------------+--------------------+---------------------+--------------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+-------------------------+--------------------+--------------------+----------------------+--------------------+----------------------+----------------------------+--------------+--------------------+--------------------+--------------------+--------------------+-------------------+----------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+--------------------+--------------------+--

In [ ]:
# Load 

sorted_reviews.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/airbnbmountdata/transformed-data/b-sortedreviews") 
result_df.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/airbnbmountdata/transformed-data/b-listings") 
